# tweetfeed

## #Data

In [1]:
import os
import pandas as pd
import sys
import re
from urllib.parse import urlparse
import numpy as np


In [2]:
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path + "/tweetfeed")
    
from data import load_tweets, clean_up_url
from twitter_utils import get_collection_list, get_collection_id

In [5]:
import tweepy
#auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(wait_on_rate_limit=True)

class Tweet(object):
    def __init__(self, embed_str=None):
        self.embed_str = embed_str

    def _repr_html_(self):
        return self.embed_str
    
def embed_str(idx):
    return api.get_oembed(idx)["html"]

In [6]:
# df = load_tweets("../home.db", days=24)
# df.to_pickle("tweetfeed.pkl")

In [7]:
df = pd.read_pickle("tweetfeed.pkl"); df.shape

(55485, 9)

In [8]:
df.head()

,id,user,full_text,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id
0,5596520,713143,soo hungry need to find my wife and head to pf...,2007-02-20T02:43:19+00:00,en,N/A,N/A,0,N/A
1,154674522,7193842,Hofstadter’s Law: A task always takes longer t...,2007-07-17T20:45:46+00:00,en,N/A,N/A,0,N/A
2,412145582,4829901,Adobe Updater must update itself before instal...,2007-11-13T21:45:35+00:00,en,N/A,N/A,0,N/A
3,922321981,16298441,no,2008-09-15T17:25:20+00:00,und,N/A,N/A,0,N/A
4,2627602600,21454322,"Went on a USO trip to Guantanamo Bay, Cuba a f...",2009-07-14T05:15:27+00:00,en,N/A,N/A,0,N/A


In [9]:
df = df.rename({"full_text" : "full_text_short"}, axis=1)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55485 entries, 0 to 55484
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     55485 non-null  int64 
 1   user                   55485 non-null  int64 
 2   full_text_short        55485 non-null  object
 3   created_at             55485 non-null  object
 4   lang                   55485 non-null  object
 5   retweeted_status       55485 non-null  object
 6   quoted_status          55485 non-null  object
 7   is_quote_status        55485 non-null  int64 
 8   in_reply_to_status_id  55485 non-null  object
dtypes: int64(3), object(6)
memory usage: 3.8+ MB


In [11]:
df.quoted_status = df.quoted_status.replace("N/A", 0).fillna(0).astype(np.int64)
df.in_reply_to_status_id = df.in_reply_to_status_id.replace("N/A", 0).fillna(0).astype(np.int64)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55485 entries, 0 to 55484
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     55485 non-null  int64 
 1   user                   55485 non-null  int64 
 2   full_text_short        55485 non-null  object
 3   created_at             55485 non-null  object
 4   lang                   55485 non-null  object
 5   retweeted_status       55485 non-null  object
 6   quoted_status          55485 non-null  int64 
 7   is_quote_status        55485 non-null  int64 
 8   in_reply_to_status_id  55485 non-null  int64 
dtypes: int64(5), object(4)
memory usage: 3.8+ MB


In [13]:
df[df.in_reply_to_status_id > 0]

,id,user,full_text_short,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id
47,667817009363611648,93278929,@ArchMaher https://twitter.com/ArchMaher/statu...,2015-11-20T21:29:17+00:00,und,N/A,0,0,667816920742129664
48,667817093123801088,93278929,@ArchMaher https://twitter.com/ArchMaher/statu...,2015-11-20T21:29:37+00:00,und,N/A,0,0,667817009363611648
62,709737027743440896,93278929,@ArchMaher #مسجد في مدينة #جدة #تصميم #معماري ...,2016-03-15T13:44:28+00:00,und,N/A,0,0,709724587823652864
68,751021335480905730,3610845615,@keffiesuplex you need to find yourself a weal...,2016-07-07T11:53:34+00:00,en,N/A,0,0,751014338895417344
71,755026645069750272,381289719,1) When ISIS was still occupying parts of Trip...,2016-07-18T13:09:14+00:00,en,N/A,0,0,755018269501841408
...,...,...,...,...,...,...,...,...,...
55473,1351678195083468800,40426297,"@collinofzion I'll take a cord of wood, thanks!",2021-01-19T23:49:29+00:00,en,N/A,0,0,1351544872738684929
55474,1351678756621082627,264043245,Dipert reckd,2021-01-19T23:51:43+00:00,en,N/A,0,0,1351678175798046723
55478,1351679374500642818,991093012804517888,@pilhuhn @mipsytipsy I got mine from https://w...,2021-01-19T23:54:10+00:00,en,N/A,0,0,1351572277196955651
55482,1351680121409728515,1717291,https://twitter.com/pkedrosky/status/135168012...,2021-01-19T23:57:08+00:00,und,N/A,0,0,1351679706379141121


In [14]:
df[df.quoted_status > 0]

,id,user,full_text_short,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id
85,811910511587717120,93278929,تكرّش حتى أمسى بلا رقبة #مظفر_النواب https://t...,2016-12-22T12:25:25+00:00,ar,N/A,811909512445788160,1,0
88,814113551778508801,3169212490,#modularity leads to #innovation @carlissbald...,2016-12-28T14:19:30+00:00,en,N/A,814110461436235776,1,0
124,877261616512892929,1575023431,المكان اللى اتصور فيه فيلم ذيب ❤ . كنت هموت وا...,2017-06-20T20:27:22+00:00,ar,N/A,876927696881799168,1,0
125,877262100606971906,93278929,@mashaly_maha https://twitter.com/archmaher/st...,2017-06-20T20:29:18+00:00,ar,N/A,667816920742129664,1,877261616512892929
200,976182659901722629,294710260,In response to this @BryanVanNorden sent me th...,2018-03-20T19:44:17+00:00,en,N/A,975771064843276288,1,0
...,...,...,...,...,...,...,...,...,...
55458,1351675122751569921,48008938,Dishonesty? Stupidity? Ignorance? Lack of fore...,2021-01-19T23:37:16+00:00,en,N/A,1351618345456594955,1,0
55460,1351675212270596097,369931522,Tying up all the loose plot lines for the fina...,2021-01-19T23:37:38+00:00,en,N/A,1351670940896284674,1,0
55464,1351676085667319814,738538436,Becoming. https://twitter.com/idaho_blue/statu...,2021-01-19T23:41:06+00:00,en,N/A,1351534278014713861,1,0
55465,1351676175794360320,17595439,After more than a century of competition and c...,2021-01-19T23:41:27+00:00,en,N/A,1351663630765469701,1,0


In [15]:
df.insert(
    3,
    "in_reply_to_text",
    df["in_reply_to_status_id"].map(df.set_index("id")["full_text_short"]),
)
df[df.in_reply_to_status_id > 0].head()

,id,user,full_text_short,in_reply_to_text,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id
47,667817009363611648,93278929,@ArchMaher https://twitter.com/ArchMaher/statu...,#photography #Jordan #Rum #وادي_رم #الأردن htt...,2015-11-20T21:29:17+00:00,und,N/A,0,0,667816920742129664
48,667817093123801088,93278929,@ArchMaher https://twitter.com/ArchMaher/statu...,@ArchMaher https://twitter.com/ArchMaher/statu...,2015-11-20T21:29:37+00:00,und,N/A,0,0,667817009363611648
62,709737027743440896,93278929,@ArchMaher #مسجد في مدينة #جدة #تصميم #معماري ...,تصميمي الداخلي لمسجد في مدينة جدة تحت التنفيذ ...,2016-03-15T13:44:28+00:00,und,N/A,0,0,709724587823652864
68,751021335480905730,3610845615,@keffiesuplex you need to find yourself a weal...,NaN,2016-07-07T11:53:34+00:00,en,N/A,0,0,751014338895417344
71,755026645069750272,381289719,1) When ISIS was still occupying parts of Trip...,NaN,2016-07-18T13:09:14+00:00,en,N/A,0,0,755018269501841408


In [16]:
df[(df.in_reply_to_status_id> 0) & (df.in_reply_to_text.isna())]

,id,user,full_text_short,in_reply_to_text,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id
68,751021335480905730,3610845615,@keffiesuplex you need to find yourself a weal...,NaN,2016-07-07T11:53:34+00:00,en,N/A,0,0,751014338895417344
71,755026645069750272,381289719,1) When ISIS was still occupying parts of Trip...,NaN,2016-07-18T13:09:14+00:00,en,N/A,0,0,755018269501841408
83,811813406281990144,93278929,@ArchMaher الشكل الخارجي والمسبح في الارتداد ا...,NaN,2016-12-22T05:59:33+00:00,ar,N/A,0,0,811170346074337282
94,830912399607672832,2895499182,This is what my desk in roppongi hills looked ...,NaN,2017-02-12T22:52:08+00:00,en,N/A,0,0,830912198696280064
96,834272217441513472,64721011,@Loganullyott @Alex_Danco MaRS is particularly...,NaN,2017-02-22T05:22:51+00:00,en,N/A,0,0,834261739566804992
...,...,...,...,...,...,...,...,...,...,...
55394,1351660372730777607,1254814620122038272,@least_nathan @encryptedcyber1 @zardigrade @Zo...,NaN,2021-01-19T22:38:40+00:00,en,N/A,0,0,1351652606926598144
55407,1351663630765469701,4808581,@kvogt https://groundtruthautonomy.com/who-wil...,NaN,2021-01-19T22:51:36+00:00,und,N/A,0,0,1351530852493127683
55426,1351668894486962177,755733097,@KordingLab @Raamana_ @graemedmoffat @tyrell_t...,NaN,2021-01-19T23:12:31+00:00,en,N/A,0,0,1351667248482693120
55474,1351678756621082627,264043245,Dipert reckd,NaN,2021-01-19T23:51:43+00:00,en,N/A,0,0,1351678175798046723


In [17]:
df.insert(
    3,
    "quoted_text",
    df["quoted_status"].map(df.set_index("id")["full_text_short"]),
)
df[df.quoted_status > 0].head()

,id,user,full_text_short,quoted_text,in_reply_to_text,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id
85,811910511587717120,93278929,تكرّش حتى أمسى بلا رقبة #مظفر_النواب https://t...,#أمجد_رسمي #كاريكاتير https://twitter.com/Rasm...,NaN,2016-12-22T12:25:25+00:00,ar,N/A,811909512445788160,1,0
88,814113551778508801,3169212490,#modularity leads to #innovation @carlissbald...,Airbus is exploring a new design that could ch...,NaN,2016-12-28T14:19:30+00:00,en,N/A,814110461436235776,1,0
124,877261616512892929,1575023431,المكان اللى اتصور فيه فيلم ذيب ❤ . كنت هموت وا...,Petra #Jordan\rمدينة البتراء في الأردن https:/...,NaN,2017-06-20T20:27:22+00:00,ar,N/A,876927696881799168,1,0
125,877262100606971906,93278929,@mashaly_maha https://twitter.com/archmaher/st...,#photography #Jordan #Rum #وادي_رم #الأردن htt...,المكان اللى اتصور فيه فيلم ذيب ❤ . كنت هموت وا...,2017-06-20T20:29:18+00:00,ar,N/A,667816920742129664,1,877261616512892929
200,976182659901722629,294710260,In response to this @BryanVanNorden sent me th...,A #Philosophy Meme https://twitter.com/lastpos...,NaN,2018-03-20T19:44:17+00:00,en,N/A,975771064843276288,1,0


In [18]:
df[(df.quoted_status> 0) & (df.quoted_text.isna())]

,id,user,full_text_short,quoted_text,in_reply_to_text,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id


In [19]:
df.insert(
    2,
    "full_text",
(df.full_text_short
    + " "
    + df.in_reply_to_text.fillna("")
    + " "
    + df.quoted_text.fillna(""))
)


In [20]:
df.drop(["full_text_short", "quoted_text", "in_reply_to_text"], axis=1, inplace=True)

In [21]:
df

,id,user,full_text,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id
0,5596520,713143,soo hungry need to find my wife and head to pf...,2007-02-20T02:43:19+00:00,en,N/A,0,0,0
1,154674522,7193842,Hofstadter’s Law: A task always takes longer t...,2007-07-17T20:45:46+00:00,en,N/A,0,0,0
2,412145582,4829901,Adobe Updater must update itself before instal...,2007-11-13T21:45:35+00:00,en,N/A,0,0,0
3,922321981,16298441,no,2008-09-15T17:25:20+00:00,und,N/A,0,0,0
4,2627602600,21454322,"Went on a USO trip to Guantanamo Bay, Cuba a f...",2009-07-14T05:15:27+00:00,en,N/A,0,0,0
...,...,...,...,...,...,...,...,...,...
55480,1351679706379141121,1717291,"Fairly bananas outflow winds, almost to the co...",2021-01-19T23:55:29+00:00,en,N/A,0,0,0
55481,1351680100408848384,1461881,I've followed @lethain’s blog for a long time ...,2021-01-19T23:57:03+00:00,en,N/A,0,0,0
55482,1351680121409728515,1717291,https://twitter.com/pkedrosky/status/135168012...,2021-01-19T23:57:08+00:00,und,N/A,0,0,1351679706379141121
55483,1351680498351939588,423622378,Here's one of mine 😭\nhttps://youtu.be/vAoADCS...,2021-01-19T23:58:38+00:00,en,N/A,0,0,1351675567842717696


### Retweets

Retweets should be removed - tweets that are retweeted are in the database, and contain full text of a tweet, icluding links and urls. This is not a case for retweets. 

They have `RT ` in front of the tweet, and are often cut at the end.

Retweets don't add anything to consversation, they just amplify signal.

In [22]:
# Retweet
df[df["id"] == 1340085773203193856].iloc[0].full_text

'RT @SophieCo_RT: Life = Matter + Information - chasing the formula of our existence with physicist and cosmologist, Prof. Paul Davies @Beyo…  '

In [23]:
# Retweeted tweet
df[df["id"] == 1339896134291091458].iloc[0].full_text

'Life = Matter + Information - chasing the formula of our existence with physicist and cosmologist, Prof. Paul Davies @Beyond_ASU: https://www.rt.com/shows/sophieco-visionaries/510041-paul-davies-life-definition/ https://twitter.com/SophieCo_RT/status/1339896134291091458/video/1  '

In [24]:
df.shape

(55485, 9)

In [25]:
# filter out retweets - retweeted tweets are in the database
# so RT only dup
df = df[df["retweeted_status"] == "N/A"] ; df.shape

(44125, 9)

## Reviewed tweets

In [26]:
reviewed_df = pd.read_csv("../tweetfeed/data/seen.csv"); reviewed_df.shape

(8460, 2)

In [27]:
reviewed_df["err_reason"].value_counts()

no_errors          6608
not_found          1680
protected_tweet     172
Name: err_reason, dtype: int64

In [28]:
deleted_tweets = reviewed_df[reviewed_df.err_reason == "not_found"].tweet_id.tolist()
df = df[~df["id"].isin(deleted_tweets)]; df.shape

(43896, 9)

In [29]:
### let's deal only with tweet that are in English

In [30]:
df = df[df["lang"] == "en"]; df.shape

(40405, 9)

## Augmentation

### Finding news related tweets

In [31]:
with open("../tweetfeed/data/news_domains.txt", "r") as f:
    news_domains = json.loads(f.read())
# df = find_news(df, news_domains); df.shape

In [34]:
df_news = df.copy()

In [35]:
def remove_tw_urls(tweet: str) -> str:
    """removes twitter links / urls from tweet"""
    tweet = re.sub(r"https://twitter.com/\S+", "", tweet)
    tweet = re.sub(r"http://twitter.com/\S+", "", tweet)
    tweet = re.sub(r"https://api.twitter.com/\S+", "", tweet)
    tweet = re.sub(r"http://api.twitter.com/\S+", "", tweet)
    return tweet

def rem_short_links(tweet: str) -> str:
    """removes some of short links (bit.ly, buff.ly, t.co) from tweets"""
    tweet = re.sub(r"https://bit.ly/\S+", "", tweet)
    tweet = re.sub(r"http://bit.ly/\S+", "", tweet)
    tweet = re.sub(r"https://buff.ly/\S+", "", tweet)
    tweet = re.sub(r"http://buff.ly/\S+", "", tweet)
    tweet = re.sub(r"https://t.co/\S+", "", tweet)
    tweet = re.sub(r"http://t.co/\S+", "", tweet)
    return tweet

In [36]:
df_news["clean_text"] = df_news["full_text"].apply(remove_tw_urls).apply(rem_short_links)

In [37]:
df_news[df_news["clean_text"].str.contains("twitter")].iloc[0].full_text

'Your twitter timeline as an interface to your mind. Someone liking your tweets can force you to have those thoughts.  '

In [38]:
df_news[df_news["clean_text"].str.contains("bit.ly")]

,id,user,full_text,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id,clean_text


In [39]:
def find_url(tweet: str) -> list:
    """find all urls in string and returns a list of all urls"""
    return re.findall(r"http\S+", tweet)

In [40]:
df_news["urls"] = df_news["clean_text"].apply(find_url) 

In [41]:
df_news.drop(["clean_text"], axis=1, inplace=True)

In [42]:
def get_domain(url: str) -> str:
    """extracts domain from url, returns it"""
    domain = urlparse(url).netloc.replace("www.", "")
    dot_split = domain.split(".")
    if (len(dot_split) > 2) & (
        dot_split[-1] == "com"
    ):  # for links like "edition.cnn.com", but not like "site.co.nz"
        return ".".join(dot_split[1:])
    else:
        return domain
def remove_empty_str(string_list: list) -> list:
    """removes items that are empty strings from the list"""
    for i in string_list:
        if len(string_list) == 0:
            string_list.remove(i)
    return string_list

In [43]:
df_news["domains"] = df_news.urls.apply(lambda x: [get_domain(d) for d in x]).apply(
    remove_empty_str
)
df_news.drop(["urls"], axis=1, inplace=True)

In [44]:
# get max value of domains, expand each one to new column (unpack from list)
new_columns_list = []
max_nr_dom = df_news.domains.str.len().max()
for i in range(max_nr_dom):
    new_columns_list.append(f"domain{i+1}")
df_news.reset_index(drop=True, inplace=True)
df_news[new_columns_list] = pd.DataFrame(df_news.domains.tolist())

for col in new_columns_list:
    df_news[col] = df_news[col].isin(news_domains)

In [45]:
df_news[["id"] + new_columns_list].head(10)

,id,domain1,domain2,domain3,domain4,domain5,domain6
0,5596520,False,False,False,False,False,False
1,154674522,False,False,False,False,False,False
2,412145582,False,False,False,False,False,False
3,2627602600,False,False,False,False,False,False
4,35122446658965504,False,False,False,False,False,False
5,35466982635601920,False,False,False,False,False,False
6,70261648811761665,False,False,False,False,False,False
7,71705067115388928,False,False,False,False,False,False
8,177008089394970624,True,False,False,False,False,False
9,193480622533120001,False,False,False,False,False,False


In [46]:
df_news[df_news.domain1 == True][["id", "domains"] + new_columns_list].head(10)

,id,domains,domain1,domain2,domain3,domain4,domain5,domain6
8,177008089394970624,[cnn.com],True,False,False,False,False,False
53,703286688064221184,[nature.com],True,False,False,False,False,False
69,814110461436235776,[bloom.bg],True,False,False,False,False,False
70,814113551778508801,[bloom.bg],True,False,False,False,False,False
81,848731022854205440,[nyti.ms],True,False,False,False,False,False
162,986941808348409856,[theguardian.com],True,False,False,False,False,False
225,1051756275585843201,[coindesk.com],True,False,False,False,False,False
241,1065383519419912193,[philly.com],True,False,False,False,False,False
257,1083458327143739392,[ft.com],True,False,False,False,False,False
298,1105013938524106757,[phys.org],True,False,False,False,False,False


In [59]:
df_news["domains_nr"] = df_news.domains.copy().apply(len)

In [64]:
df_news.sort_values(by=["domains_nr"], ascending=False).head()

,id,user,full_text,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id,domains,domain1,domain2,domain3,domain4,domain5,domain6,domains_nr
3399,1335970579313287174,14403154,🥳 Finally out x2: HyPyP Beta released + publis...,2020-12-07T15:33:01+00:00,en,N/A,1263439803305136128,1,0,"[github.com, hypyp.readthedocs.io, oup.com, gi...",False,False,True,False,False,False,6
24957,1344990732780711937,731538535795163136,Growing body of work on how to better treat th...,2021-01-01T12:55:54+00:00,en,N/A,1344730628407881729,1,0,"[arxiv.org, arxiv.org, arxiv.org, bair.berkele...",False,False,False,False,False,False,6
36105,1349774302149660672,224896427,The variant emerging in Brazil (@nextstrain cl...,2021-01-14T17:44:05+00:00,en,N/A,0,0,1349774296319549442,"[cov-lineages.org, virological.org, nextstrain...",False,False,False,False,False,False,6
24253,1344730628407881729,928922976728289281,"Finally, inspiring works really worth a read:\...",2020-12-31T19:42:20+00:00,en,N/A,0,0,1344730626755158016,"[arxiv.org, arxiv.org, arxiv.org, bair.berkele...",False,False,False,False,False,False,6
7043,1336857186350354432,2465283662,Improving the Fairness of Deep Generative Mode...,2020-12-10T02:16:05+00:00,en,N/A,0,0,0,"[arxiv.org, arxiv.org, genforce.github.io, git...",False,False,False,False,False,False,5


In [47]:
df_news[df_news.domain4 == True][["id", "domains"] + new_columns_list].head(10)

,id,domains,domain1,domain2,domain3,domain4,domain5,domain6
8978,1337449293834113025,"[simplecast.com, dailymail.co.uk, insider.com,...",False,True,False,True,True,False
14187,1339626801513996293,"[cargo.one,, Tech.eu), tech.eu, techmeme.com]",False,False,False,True,False,False
30316,1347533968212688896,"[fbi.gov, fbi.gov, fbi.gov, fbi.gov]",True,True,True,True,False,False
30459,1347560416323358725,"[fbi.gov, fbi.gov, fbi.gov, fbi.gov]",True,True,True,True,False,False


In [48]:
df[df["id"] == 1337449293834113025].iloc[0].full_text

"So, @morewoke...we see you're ALSO from #SantaFeNM...coincidence? 🧐\n\n🛸🕋 #ComplexityPodcast\nhttps://complexity.simplecast.com\n\n@DailyMailUK\nhttps://www.dailymail.co.uk/news/article-9022119/New-Mexico-artist-collective-claims-mysterious-steel-monoliths.html\n\n@thisisinsider\nhttps://www.insider.com/who-made-the-monolith-artist-collective-selling-45k-2020-12\n\n@hyperallergic\nhttps://hyperallergic.com/606128/monolith-mania-continues-as-artists-come-forward-to-claim-credit-for-sculptures/ https://twitter.com/sfiscience/status/1337449293834113025/photo/1 We all agree that the only thing left for the 2020 news cycle is alien disclosure, but come on, y'all. 2020 is 19 years late.\n\nWe've had our alpha & beta monoliths for...how long, now?\n\nLost track of time with all the dry-erase marker glyphs flying about.\n\nhttps://www.kob.com/albuquerque-news/mysterious-monolith-spotted-in-albuquerque-/5944452/#.X86A0_40M_c.facebook https://twitter.com/sfiscience/status/1336089616466784256/ph

In [50]:
df_news[df_news["id"] == 1337449293834113025].iloc[0].domains

['simplecast.com',
 'dailymail.co.uk',
 'insider.com',
 'hyperallergic.com',
 'kob.com']

In [45]:
most_links = (
    df_news.domains.apply(lambda x: len(x)).sort_values(ascending=False).index[0]
) #get index of tweet with most links

In [46]:
Tweet(embed_str(df_news.iloc[most_links]["id"]))

In [47]:
df_news.drop(["domains"], axis=1, inplace=True)
# sum it all up
df_news["contains_news"] = df_news[new_columns_list].sum(axis=1)

In [48]:
df_news["contains_news"].value_counts()

0    37401
1     2305
2      716
3        5
4        2
Name: contains_news, dtype: int64

In [49]:
# we are interest in only a True-False, so let's change that.
df_news["contains_news"] = df_news.contains_news.apply(lambda x: x if x == 0 else 1)
# now we can remove added columns
df_news.drop(new_columns_list, axis=1, inplace=True)

In [50]:
df_news["contains_news"].value_counts()

0    37401
1     3028
Name: contains_news, dtype: int64

In [51]:
def find_news(df: pd.DataFrame, news_domains_list: list) -> pd.DataFrame:
    """Takes DataFrame, and list of domains of news sites.
    Removes from DataFrame rows that contain links to sites from that list.
    Args:
        df (pd.DataFrame): DataFrame to be cleaned
        news_domains_list (list): list of domains of news sites

    Returns:
        pd.DataFrame: DataFrame without tweets linking to news
    """
    df = df.copy()
    df["clean_text"] = (
        df["full_text"].apply(remove_tw_urls).apply(rem_short_links)
    )
    df["clean_text"] = df["clean_text"].apply(rem_short_links)
    df["urls"] = df["clean_text"].apply(find_url)
    df.drop(["clean_text"], axis=1, inplace=True)
    df["domains"] = df.urls.apply(lambda x: [clean_up_url(d) for d in x])
    df["domains"] = df.urls.apply(lambda x: [get_domain(d) for d in x])
    df["domains"] = df.domains.apply(remove_empty_str)
    df.drop(["urls"], axis=1, inplace=True)

    # get max value of domains, expand each one to new column (unpack from list)
    new_columns_list = []
    max_nr_dom = df.domains.str.len().max()
    for i in range(max_nr_dom):
        new_columns_list.append(f"domain{i+1}")
    df.reset_index(drop=True, inplace=True)
    df[new_columns_list] = pd.DataFrame(df.domains.tolist())

    for col in new_columns_list:
        df[col] = df[col].isin(news_domains_list)

    df.drop(["domains"], axis=1, inplace=True)
    # sum it all up
    df["contains_news"] = df[new_columns_list].sum(axis=1)
    df["contains_news"] = df.contains_news.apply(lambda x: x if x == 0 else 1)
    # remove added columns
    df.drop(new_columns_list, axis=1, inplace=True)

    return df

In [52]:
df = find_news(df, news_domains); df.shape

(40429, 10)

In [53]:
df_news["contains_news"].value_counts()

0    37401
1     3028
Name: contains_news, dtype: int64

## explore muted accounts, and muted words

In [54]:
with open("../tweetfeed/data/mute_list.txt", "r") as f:
    mute_list = json.loads(f.read())
with open("../tweetfeed/data/mute_list_cs.txt", "r") as f:
    mute_list_cs = json.loads(f.read())
with open("../tweetfeed/data/mutedacc.txt", "r") as f:
    mutedacc = json.loads(f.read())

In [55]:
mute_list_cs

['GOP', 'BREAKING', 'BLM', 'MAGA', 'Rep.']

In [56]:
mute_list[:4]

['breaking:', '🍿', '🚨', 'New Yorker']

In [57]:
mutedacc[:5]

[1282696358034657281,
 1242108820362530820,
 1168694596450299906,
 1116562680607846400,
 1093986843648016384]

In [58]:
df_mute = df.copy()
m_list = mute_list
column_name = "full_text"
for item in m_list:
    df_mute[item] = df_mute[column_name].str.contains(item)
df_mute["is_mute_list"] = df_mute[mute_list].sum(axis=1)
df_mute["is_mute_list"] = df_mute["is_mute_list"].apply(lambda x: x if x == 0 else 1)
df_mute.drop(mute_list, axis=1, inplace=True)
df_mute[df_mute.is_mute_list > 0].shape

(724, 11)

In [59]:
def is_muted(df, mute_list, column_name, new_col, case=True):
    df_mute = df.copy()
    for item in mute_list:
        df_mute[item] = df_mute[column_name].str.contains(item, case)
    df_mute[new_col] = df_mute[mute_list].sum(axis=1)
    df_mute[new_col] = df_mute[new_col].apply(lambda x: x if x == 0 else 1)
    df_mute.drop(mute_list, axis=1, inplace=True)
    return df_mute

In [60]:
df = is_muted(df, mute_list, "full_text", "is_mute_list", case=False)
df = is_muted(df, mute_list_cs, "full_text", "is_mute_list_cs")
df["mutedacc"] = np.where(df["user"].isin(mutedacc), 1, 0)

In [61]:
columns_list = ["is_mute_list", "is_mute_list_cs", "mutedacc"]
df["muted"] = (
    df[columns_list]
    .sum(axis=1)
    .apply(lambda x: x if x == 0 else 1)
)
df.drop(columns_list, axis=1, inplace=True)

In [62]:
df[["id", "user", "contains_news", "muted"]].head(10)

,id,user,contains_news,muted
0,5596520,713143,0,0
1,154674522,7193842,0,0
2,412145582,4829901,0,0
3,2627602600,21454322,0,0
4,35122446658965504,5691532,0,0
5,35466982635601920,15012642,0,0
6,70261648811761665,5416652,0,0
7,71705067115388928,14182218,0,0
8,177008089394970624,5110861,1,0
9,193480622533120001,175624200,0,0


In [63]:
df[(df.contains_news > 0) | (df.muted > 0)].shape #.copy and put into new dataframe

(8720, 11)

In [64]:
reviewed_df = pd.read_csv("../tweetfeed/data/seen.csv"); reviewed_df.shape

(7740, 2)

In [65]:
Tweet(embed_str("177008089394970624"))

In [66]:
seen = reviewed_df[reviewed_df.err_reason == "no_errors"].tweet_id.tolist(); len(seen)

5951

In [67]:
df[df.id.isin(seen) & (df.contains_news > 0) & (df.muted == 0)]

,id,user,full_text,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id,contains_news,muted
535,1213897615273844736,1269670536,One actual response option to cyberattacks mig...,2020-01-05T18:58:39+00:00,en,N/A,1213526811935686656,1,0,1,0
1559,1330554505612521473,466880049,NYT is running auditions for a new post-inaugu...,2020-11-22T16:51:29+00:00,en,N/A,1329794087684345859,1,0,1,0
1572,1330557102390009857,1416500532,"IIRC, this was the premise of a Greg Bear two-...",2020-11-22T17:01:48+00:00,en,N/A,1330342856054202368,1,0,1,0
1706,1330599988229857283,8500962,"Hmm. So by the time we have normalcy, the US w...",2020-11-22T19:52:13+00:00,en,N/A,1330596971011133441,1,0,1,0
1829,1330664128810086400,894970070358564864,the math classes i learned the most from in co...,2020-11-23T00:07:05+00:00,en,N/A,1330513410673487873,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
36077,1349769501999554564,420308365,Big tech will soon realize that exporting extr...,2021-01-14T17:25:01+00:00,en,N/A,1349750205801922561,1,0,1,0
36167,1349777245406113792,2462430336,The dumbest thing about waited priority is sim...,2021-01-14T17:55:47+00:00,en,N/A,0,0,1349776763694542850,1,0
37425,1350152689045405698,1480749782,"Turkey vaccinates over 500,000 in two days ht...",2021-01-15T18:47:40+00:00,en,N/A,0,0,0,1,0
38063,1350940067863392259,44880062,"Worth watching. \n""How long will you continue ...",2021-01-17T22:56:26+00:00,en,N/A,1350367953942937601,1,0,1,0


In [68]:
Tweet(embed_str("1330677864555896834"))

In [69]:
# I remember seeing this tweet, and  then I added `tvnz.co.nz` to domains containing news

In [70]:
auth = "../auth/auth.json"
owner_id = "143058191"

In [71]:
not_relevant_list = get_collection_list(
    get_collection_id(owner_id, "not_relevant", auth), auth
)

In [72]:
df["not_relevant_list"] = np.where(df["id"].isin(not_relevant_list), 1, 0)

In [73]:
df[df.not_relevant_list > 0]

,id,user,full_text,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id,contains_news,muted,not_relevant_list
1587,1330561509576093696,150281740,Why... why would the kids even think about thi...,2020-11-22T17:19:19+00:00,en,N/A,1330560915616526339,1,0,0,1,1
1706,1330599988229857283,8500962,"Hmm. So by the time we have normalcy, the US w...",2020-11-22T19:52:13+00:00,en,N/A,1330596971011133441,1,0,1,0,1
2142,1333820146125852673,1232294491589152775,"Name a character who went through more pain, i...",2020-12-01T17:07:58+00:00,en,N/A,0,0,0,0,0,1
2230,1334904553700028418,4827186013,So what is the best way to keep students on tr...,2020-12-04T16:57:01+00:00,en,N/A,0,0,0,0,1,1
2526,1335662890054000641,63506279,@Meaningness ya same!! @alicemazzy Oh THAT Men...,2020-12-06T19:10:23+00:00,en,N/A,0,0,1335662190255169537,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
31089,1347922375925501954,309822757,"No. 2 House Republican Rep. Steve Scalise: ""I ...",2021-01-09T15:05:12+00:00,en,N/A,0,0,0,0,1,1
31505,1348076274699350018,20878297,"MARTIAL LAW: The province of Quebec, Canada is...",2021-01-10T01:16:44+00:00,en,N/A,0,0,0,0,0,1
32630,1348629770397908993,817386,"Sources: China will push tech giants, includin...",2021-01-11T13:56:08+00:00,en,N/A,0,0,0,1,1,1
34391,1349041878273044481,925124076666007558,#Datanova2021 is less than a month away! Our t...,2021-01-12T17:13:42+00:00,en,N/A,0,0,0,0,0,1


In [74]:
df[df.not_relevant_list > 0][["contains_news", "muted", "not_relevant_list"]].sum()

contains_news         19
muted                 36
not_relevant_list    131
dtype: int64

## Exploratory Data Analysis (EDA)

In [75]:
# which accounts post most news?
# which accounts post most?




## Annotation

## Labeling
## Spliting
## Preprocessing


# Modeling
## Random
## Rule-based
## Simple ML
## CNN
## RNN
## Transformers


# Experiment tracking
## Training
## Tracking
## Viewing
## Loading

# Optimization